# 08 — Feed Forward Network

A simple Feed Forward Neural Network (FFN) using **Word2Vec document vectors** as input.

**Architecture**: `100 → 64 → 32 → 1` (ReLU + Dropout + Sigmoid)

Trained on **Standard**, **Irony**, and **Obfuscated** pipelines.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report
import os
# Corpus Configuration
CORPUS_NAME = 'raw-corpus' # Options: 'pre-filtered-corpus', 'raw-corpus'
PROCESSED_DATA_DIR = f'../data/processed/{CORPUS_NAME}'
MODELS_DIR_BASE = f'../models/{CORPUS_NAME}'



In [2]:
%load_ext watermark
%watermark -v -n -m -p numpy,torch,sklearn

Python implementation: CPython
Python version       : 3.12.12
IPython version      : 9.10.0

numpy  : 1.26.4
torch  : 2.2.2
sklearn: 1.8.0

Compiler    : Clang 17.0.0 (clang-1700.6.3.2)
OS          : Darwin
Release     : 25.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



## 1. Model Definition

In [3]:
class FFN(nn.Module):
    def __init__(self, input_dim=100):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

## 2. Training Function

In [4]:
def train_ffn(variation_name, w2v_dir, output_dir, epochs=20, lr=1e-3, batch_size=32):
    print(f"\n{'='*20} FFN: {variation_name} {'='*20}")
    
    # Load pre-computed Word2Vec document vectors
    X_train = np.load(f'{w2v_dir}/doc_vectors_train.npy', allow_pickle=True)
    X_test  = np.load(f'{w2v_dir}/doc_vectors_test.npy', allow_pickle=True)
    labels_train = np.load(f'{w2v_dir}/labels_train.npy', allow_pickle=True)
    label_map = {'NEGATIVE': 0, 'POSITIVE': 1}
    y_train = np.array([label_map[l] for l in labels_train]).astype(np.float32)
    labels_test = np.load(f'{w2v_dir}/labels_test.npy', allow_pickle=True)
    y_test = np.array([label_map[l] for l in labels_test]).astype(np.float32)
    
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
    
    # PyTorch datasets
    train_ds = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(y_train)
    )
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    
    # Model
    model = FFN(input_dim=X_train.shape[1])
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Train
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            output = model(batch_x).squeeze()
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{epochs} — Loss: {total_loss/len(train_loader):.4f}")
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        X_test_t = torch.FloatTensor(X_test)
        preds = model(X_test_t).squeeze()
    y_pred = np.array((preds >= 0.5).int().tolist())
    
    acc = accuracy_score(y_test, y_pred)
    print(f"\nFFN ({variation_name}) Accuracy: {acc:.4f}")
    print(classification_report(y_test.astype(int), y_pred))
    
    # Save
    os.makedirs(output_dir, exist_ok=True)
    torch.save(model.state_dict(), f'{output_dir}/model.pt')
    print(f"Model saved to {output_dir}/model.pt")
    
    return acc

## 3. Run All Pipelines

In [5]:
acc_standard = train_ffn("Standard", f"{MODELS_DIR_BASE}/word2vec/standard", f"{MODELS_DIR_BASE}/ffn/standard")

acc_irony = train_ffn("Irony", f"{MODELS_DIR_BASE}/word2vec/irony", f"{MODELS_DIR_BASE}/ffn/irony")

acc_obfuscated = train_ffn("Obfuscated", f"{MODELS_DIR_BASE}/word2vec/obfuscated", f"{MODELS_DIR_BASE}/ffn/obfuscated")



==================== FFN: Standard ====================
Train: (2100, 100), Test: (450, 100)


  Epoch 5/20 — Loss: 0.5211


  Epoch 10/20 — Loss: 0.5027


  Epoch 15/20 — Loss: 0.4786


  Epoch 20/20 — Loss: 0.4637

FFN (Standard) Accuracy: 0.7600
              precision    recall  f1-score   support

           0       0.73      0.83      0.78       225
           1       0.80      0.69      0.74       225

    accuracy                           0.76       450
   macro avg       0.77      0.76      0.76       450
weighted avg       0.77      0.76      0.76       450

Model saved to ../models/raw-corpus/ffn/standard/model.pt

==================== FFN: Irony ====================
Train: (2100, 100), Test: (450, 100)


...

  Epoch 20/20 — Loss: 0.4645

FFN (Irony) Accuracy: 0.7689
              precision    recall  f1-score   support

           0       0.74      0.82      0.78       225
           1       0.80      0.72      0.76       225

    accuracy                           0.77       450
   macro avg       0.77      0.77      0.77       450
weighted avg       0.77      0.77      0.77       450

Model saved to ../models/raw-corpus/ffn/irony/model.pt

==================== FFN: Obfuscated ====================
Train: (2100, 100), Test: (450, 100)


  Epoch 5/20 — Loss: 0.5374


  Epoch 10/20 — Loss: 0.5043


  Epoch 15/20 — Loss: 0.4788


  Epoch 20/20 — Loss: 0.4615

FFN (Obfuscated) Accuracy: 0.7333
              precision    recall  f1-score   support

           0       0.69      0.85      0.76       225
           1       0.80      0.62      0.70       225

    accuracy                           0.73       450
   macro avg       0.75      0.73      0.73       450
weighted avg       0.75      0.73      0.73       450

Model saved to ../models/raw-corpus/ffn/obfuscated/model.pt


## 4. Comparison

In [6]:
print("\n=== Final Comparison ===")
print(f"Standard: {acc_standard:.4f}")
print(f"Irony:    {acc_irony:.4f}")
print(f"Obfuscated: {acc_obfuscated:.4f}")
diff = acc_irony - acc_standard
print(f"Impact of Irony features: {diff:+.4f}")


=== Final Comparison ===
Standard: 0.7600
Irony:    0.7689
Obfuscated: 0.7333
Impact of Irony features: +0.0089
